In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pk
#import networkx as nx
import re 

path = "c:/Facultad/Tesis/"
base = pd.read_csv(path+'Corpus_medios_nac.csv')
#base2 = pd.read_csv(path+'Corpus_medios_nac.csv', nrows=20, skiprow
base

C:\Users\Asus\AppData\Local\Temp\ipykernel_12424\255936669.py:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  base = pd.read_csv(path+'Corpus_medios_nac.csv')


,link,medio,provincia,seccion,fecha,hora,autor,volanta,titulo,subtitulo,nota,tags
0,https://www.ambito.com/edicion-impresa/afip-le...,ambito,Buenos Aires,economia,2018/11/08,NaN,carlos lamiral,,AFIP le hará al contribuyente la declaración...,MEDIDAS PARA FACILITAR EL PAGO DE IMPUESTOS - ...,Por Carlos Lamiral En la Administración Fe...,
1,https://www.ambito.com/edicion-impresa/bcra-cu...,ambito,Buenos Aires,economia,2018/11/08,NaN,jorge g. herrera,,BCRA cumplió gracias a la compra de dólares ...,ASÍ LOGRÓ APROBAR EL PRIMER MES DE LA META M...,Por Jorge G. Herrera Tras el cierre de octu...,
2,https://www.ambito.com/edicion-impresa/dialogo...,ambito,Buenos Aires,economia,2018/11/08,NaN,josé siaba serrate,,Diálogos de Wall Street,Día de examen para la administración Trump. ...,Por José Siaba Serrate Periodista: Un país...,
3,https://www.ambito.com/edicion-impresa/falta-e...,ambito,Buenos Aires,economia,2018/11/08,NaN,horacio f. lópez,,Falta equidad en el trato al trabajador autónomo,NaN,Por Horacio F. López En 1932 nace la Ley ...,
4,https://www.ambito.com/edicion-impresa/riesgo-...,ambito,Buenos Aires,economia,2018/11/08,NaN,caetano mohorade,,Riesgo-país perforó los 600 puntos (cerró e...,Fue la primera vez desde que el BCRA comenzó ...,Por Caetano Mohorade El riesgo-país finali...,
...,...,...,...,...,...,...,...,...,...,...,...,...
476920,https://www.losprimeros.tv/nota/politica/26914...,losprimerostv,Tucuman,politica,2021/07/21,NaN,none,,El lunes 26 de Julio comenzará el operativo d...,,El próximo lunes 26 de Julio iniciará en Tuc...,
476921,https://www.losprimeros.tv/nota/politica/26913...,losprimerostv,Tucuman,politica,2021/07/21,NaN,none,,Alberto Fernández puso en marcha DNI no binar...,,El presidente Alberto Fernández anunció este...,"['Alberto Fernández', 'dni']"
476922,https://www.losprimeros.tv/nota/politica/26913...,losprimerostv,Tucuman,politica,2021/07/21,NaN,none,,Santiago Cafiero fue papá por cuarta vez: Nac...,,"El jefe de Gabinete de Ministros, Santiago Caf...",['Santiago Cafiero']
476923,https://www.losprimeros.tv/nota/politica/26909...,losprimerostv,Tucuman,politica,2021/07/21,NaN,none,,"""Juntos para Construir"": El alfarismo también...",,El alfarismo se suma a la carrera electoral de...,"['PASO', 'german alfaro', 'Elecciones nacional..."


In [3]:
def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    
    return x

def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def words_length(x):
    return len(x.split(" "))

def pre_process_quoted(df):
    df['nota'] = df['nota'].apply(replace)
    df['citas']  = df['nota'].apply(extract_quotes)
    df_quoutes2 = pd.DataFrame({'Fecha': df['fecha'].iloc[1:], 'Hora': df['hora'].iloc[1:], 'Cita': df['citas'].iloc[1:], 'Link': df['link'].iloc[1:], 'Nota': df['nota'].iloc[1:]})
    df_quoutes = df_quoutes2.explode('Cita') # Cada cita sea una fila distinta
    df_quoutes = df_quoutes.dropna(subset=['Cita']) # Elimino la filas que no haya quotes

    #df_quoutes['Citas'] = df_quoutes['Citas'].apply(replace)

    df_quoutes = df_quoutes.reset_index()
    df_quoutes = df_quoutes.drop(['index'], axis = 1)
    df_quoutes['Cant_Palabras'] = df_quoutes['Cita'].apply(words_length) # genero columna que cuente la cantidad de plabaras
    df_quoutes_pf = df_quoutes[df_quoutes['Cant_Palabras'] > 4]
    df_quoutes_pf = df_quoutes_pf[df_quoutes_pf['Cant_Palabras'] < 60]
    df_quoutes_pf = df_quoutes_pf.reset_index()
    df_quoutes_pf = df_quoutes_pf.drop(['index'], axis = 1)
    return df_quoutes_pf

In [4]:
base['fecha'] = pd.to_datetime(base['fecha'])
fecha_inicio = pd.to_datetime('2019-01-01')
fecha_fin = pd.to_datetime('2019-04-01')

tres_meses = base[(base['fecha'] >= fecha_inicio) & (base['fecha'] <= fecha_fin)]
filt_tres_meses = pre_process_quoted(tres_meses)


C:\Users\Asus\AppData\Local\Temp\ipykernel_12424\4064238392.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nota'] = df['nota'].apply(replace)
C:\Users\Asus\AppData\Local\Temp\ipykernel_12424\4064238392.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['citas']  = df['nota'].apply(extract_quotes)


In [5]:
pathGit = 'c:/Git_Proyects/Tesis/Base_Medios_Arg/'
clusters = pk.load(open(pathGit+'clusters_threshold_75_nac_3_meses.pk', 'rb'))
frases = pk.load(open(pathGit+'nac_3_meses.pk', 'rb'))

In [6]:
cluste_map = {frases[j]: str(i+1) for i, clus in enumerate(clusters) for j in clus}

In [7]:
filt_tres_meses['id_cluster'] = filt_tres_meses['Cita'].map(cluste_map)
filt_tres_meses.dropna(subset=['id_cluster'])

,Fecha,Hora,Cita,Link,Nota,Cant_Palabras,id_cluster
0,2019-01-02,NaN,esta actualización favorece mucho más a la m...,https://www.ambito.com/edicion-impresa/empleo/...,Las empresas contarán desde este 1 de enero c...,44,7092
1,2019-01-02,NaN,una probabilidad acumulada del 50% de que la r...,https://www.ambito.com/edicion-impresa/alemani...,Por Alejandro Tagliavini Visto que las empre...,14,17182
2,2019-01-02,NaN,para tomar decisiones de política monetaria y...,https://www.ambito.com/edicion-impresa/banco-c...,Por Alejandro Colle Esta semana se conocera...,8,17183
3,2019-01-02,NaN,"al REM enviamos el escenario base, el que cons...",https://www.ambito.com/edicion-impresa/banco-c...,Por Alejandro Colle Esta semana se conocera...,26,17184
4,2019-01-02,NaN,Las empresas nos piden forzar escenarios de st...,https://www.ambito.com/edicion-impresa/banco-c...,Por Alejandro Colle Esta semana se conocera...,21,17185
...,...,...,...,...,...,...,...
121736,2019-04-01,NaN,El candidato a presidente de Cambiemos es el p...,http://www.losprimeros.tv/nota/92025/candidato...,"""El candidato a presidente de Cambiemos es el ...",18,152
121737,2019-04-01,NaN,Nadie está dejando el Gobierno. En el caso de...,http://www.losprimeros.tv/nota/92025/candidato...,"""El candidato a presidente de Cambiemos es el ...",37,2102
121738,2019-04-01,NaN,siendo el jefe de bloque y lo va a ser hasta e...,http://www.losprimeros.tv/nota/92025/candidato...,"""El candidato a presidente de Cambiemos es el ...",15,2991
121739,2019-04-01,NaN,"Lo mismo ocurre conmigo, voy a seguir al lado ...",http://www.losprimeros.tv/nota/92025/candidato...,"""El candidato a presidente de Cambiemos es el ...",15,1759


In [12]:
filt_tres_meses[filt_tres_meses['id_cluster'] == '1']


,Fecha,Hora,Cita,Link,Nota,Cant_Palabras,id_cluster
87625,2019-03-31,02:01,: Eduardo Beatrice 1060 Precandidatos a inten...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",16,1
87633,2019-03-31,02:01,: Adriana Aguirre 349 Precandidatos a intende...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",15,1
87646,2019-03-31,02:01,: Roberto Carlos Enrique 1023 Precandidatos a...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",16,1
87653,2019-03-31,02:01,: Ricardo Colomer 745 Precandidatos a intende...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",15,1
87657,2019-03-31,02:01,: Ricardo Trigo 28 Precandidatos a intendente...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",16,1
...,...,...,...,...,...,...,...
103898,2019-03-31,02:01,: Yésica Cabrera 168 Precandidatos a intende...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",15,1
103905,2019-03-31,02:01,: Yésica Cabrera 168 Precandidatos a intende...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",15,1
103911,2019-03-31,02:01,: Jorge Palacio 9 Yésica Cabrera 168 Pre...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",22,1
103917,2019-03-31,02:01,: Jorge Palacio 9 Precandidatos a intende...,https://www.tiempodesanjuan.com//politica/2019...,"Casi cerrados los comicios, mirá cuántos vot...",19,1


In [58]:
filt_tres_meses['id_cluster']

0          7092
1         17182
2         17183
3         17184
4         17185
          ...  
121736      152
121737     2102
121738     2991
121739     1759
121740    15487
Name: id_cluster, Length: 121741, dtype: object